In [1]:
import pandas as pd
import time
import ray
from oanda_forex_utilities import *
import os
import warnings



/opt/anaconda3/lib/python3.8/site-packages/ray/autoscaler/_private/cli_logger.py:57: FutureWarning: Not all Ray CLI dependencies were found. In Ray 1.4+, the Ray CLI, autoscaler, and dashboard will only be usable via `pip install 'ray[default]'`. Please update your install command.
  warnings.warn(


In [2]:
account_details = get_oanda_account_credentials("demo_account_1.14")
account_id = account_details["account_id"]
access_token = account_details["access_token"]

print(account_id)
print(access_token)

101-003-18483320-017
87a216bb7b0e640e4af97a43499b58b9-4803472d0b66dd35b5d7541f3105caf5


In [3]:

ray.init()
@ray.remote
def automated_trade(currency_pair):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')  
        get_extraction_time("M5")
        time.sleep(1.5)
        state = True
        complete_list = []

        while state == True:
            df = get_candle_data(account_id, access_token, currency_pair, "M5", 500)
            df_longer_time = get_candle_data(account_id, access_token, currency_pair, "M30", 500)

            df_with_indicators = get_macd_indicator(df)
            df_with_indicators = get_adx_indicator(df_with_indicators)
            df_with_indicators_complete = df_with_indicators[df_with_indicators["complete"] == True]

            df_longer_time_with_indicators = get_ema_indicator(df_longer_time, window = 8)

            if df_with_indicators_complete["time"].iloc[-1] not in complete_list:
                complete_list.append(df_with_indicators_complete["time"].iloc[-1])

                if (
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-1] < 0) and
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-2] < 0) and      
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-3] >= 0) and
                        (df_with_indicators_complete["adx"].iloc[-1] >= 20) and
                        (df_with_indicators_complete["time"].iloc[-1].time() >= datetime.time(6, 00)) and
                        (df_with_indicators_complete["time"].iloc[-1].time() <= datetime.time(18, 00)) and
                        (df_longer_time_with_indicators["price_close"].iloc[-1] < df_longer_time_with_indicators["ema_8"].iloc[-1])


                    ):
                    print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " SELL ORDER FILLED")
                    if currency_pair in ["EUR_JPY", "AUD_JPY", "NZD_JPY", "USD_JPY", "GBP_JPY"]:
                        stop_loss = 1000
                    else:
                        stop_loss = 10
        #             create_market_order_with_sl_tp(account_id, access_token, currency_pair, "SELL", 5000, 10, 10)
                    create_market_order_with_sl_tp(account_id, access_token, currency_pair, "SELL", 5000,
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) + stop_loss/10000,5),
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) - stop_loss/10000,5))


                elif(
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-1] > 0) and
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-2] > 0) and
                        (df_with_indicators_complete["macd_signal_diff"].iloc[-3] <= 0) and
                        (df_with_indicators_complete["adx"].iloc[-1] >= 20) and
                        (df_with_indicators_complete["time"].iloc[-1].time() >= datetime.time(6, 00)) and
                        (df_with_indicators_complete["time"].iloc[-1].time() <= datetime.time(18, 00)) and
                        (df_longer_time_with_indicators["price_close"].iloc[-1] > df_longer_time_with_indicators["ema_8"].iloc[-1])
                    ):
                    print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " BUY ORDER FILLED")
                    if currency_pair in ["EUR_JPY", "AUD_JPY", "NZD_JPY", "USD_JPY", "GBP_JPY"]:
                        stop_loss = 1000
                    else:
                        stop_loss = 10
        #             create_market_order_with_sl_tp(account_id, access_token, currency_pair, "SELL", 5000, 10, 10)
                    create_market_order_with_sl_tp(account_id, access_token, currency_pair, "BUY", 5000,
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) - stop_loss/10000,5),
                                              round(float(df_with_indicators_complete["price_close"].iloc[-1]) + stop_loss/10000,5))

                else:
                    print(df_with_indicators_complete["time"].iloc[-1], currency_pair, " NOTHING HAPPENS")
            time.sleep(10)

2021-08-02 21:36:34,614	INFO services.py:1245 -- View the Ray dashboard at http://127.0.0.1:8265


In [4]:
for i in range(1, 10):
    try:
        ray.get([automated_trade.remote("EUR_USD"), automated_trade.remote("USD_JPY"),
                 automated_trade.remote("USD_CHF"), automated_trade.remote("GBP_USD"),
                 automated_trade.remote("GBP_CHF"), automated_trade.remote("GBP_JPY"),
                 automated_trade.remote("AUD_USD"), automated_trade.remote("EUR_GBP")])
    except Exception as e:
        print (e)
        print ('Restarting!')
        ray.shutdown()
        continue
    else:
        break

(pid=46115) 2021-08-02 13:35:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46113) 2021-08-02 13:35:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46108) 2021-08-02 13:35:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46114) 2021-08-02 13:35:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46109) 2021-08-02 13:35:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46110) 2021-08-02 13:35:00+00:00 AUD_USD  BUY ORDER FILLED
(pid=46112) 2021-08-02 13:30:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46111) 2021-08-02 13:35:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 13:35:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46111) 2021-08-02 13:40:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 13:40:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46109) 2021-08-02 13:40:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46114) 2021-08-02 13:40:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46115) 2021-08-02 13:40:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46108) 2021-08-02 13:40:00+00:00 GBP_CHF  BUY ORDER FILLED
(pid=46113) 2021-08-02 13:40:00+00:00 EUR_GBP  NOTHIN

(pid=46108) 2021-08-02 14:40:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46111) 2021-08-02 14:40:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46115) 2021-08-02 14:40:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46110) 2021-08-02 14:40:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 14:40:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46109) 2021-08-02 14:45:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46108) 2021-08-02 14:45:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46111) 2021-08-02 14:45:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46115) 2021-08-02 14:45:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46110) 2021-08-02 14:45:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 14:45:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46112) 2021-08-02 14:45:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46114) 2021-08-02 14:45:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 14:50:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46114) 2021-08-02 14:50:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46109) 2021-08-02 14:50:00+00:00 GBP_JPY  NOTHING 

(pid=46112) 2021-08-02 15:45:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46115) 2021-08-02 15:50:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46110) 2021-08-02 15:50:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 15:50:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46109) 2021-08-02 15:50:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46114) 2021-08-02 15:50:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 15:50:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46111) 2021-08-02 15:50:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46108) 2021-08-02 15:50:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46114) 2021-08-02 15:55:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 15:55:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46111) 2021-08-02 15:55:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46108) 2021-08-02 15:55:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46110) 2021-08-02 15:55:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 15:55:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46109) 2021-08-02 15:55:00+00:00 GBP_JPY  NOTHING 

(pid=46110) 2021-08-02 16:55:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 16:55:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46114) 2021-08-02 16:55:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 16:55:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46115) 2021-08-02 17:00:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46108) 2021-08-02 17:00:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46110) 2021-08-02 17:00:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46109) 2021-08-02 17:00:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46113) 2021-08-02 17:00:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46111) 2021-08-02 17:00:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46114) 2021-08-02 17:00:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 17:00:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46108) 2021-08-02 17:05:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46109) 2021-08-02 17:05:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46110) 2021-08-02 17:05:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 17:05:00+00:00 EUR_GBP  NOTHING 

(pid=46112) 2021-08-02 18:05:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46113) 2021-08-02 18:05:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46111) 2021-08-02 18:05:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46114) 2021-08-02 18:05:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46115) 2021-08-02 18:05:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46109) 2021-08-02 18:05:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46108) 2021-08-02 18:05:00+00:00 GBP_CHF 
(pid=46108)  NOTHING HAPPENS
(pid=46110) 2021-08-02 18:05:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46109) 2021-08-02 18:10:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46108) 2021-08-02 18:10:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46110) 2021-08-02 18:10:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 18:10:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46111) 2021-08-02 18:10:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 18:10:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46114) 2021-08-02 18:10:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46115) 2021-08-02 18:10:00+00:00 USD_

...

(pid=46113) 2021-08-02 20:20:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46111) 2021-08-02 20:20:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46110) 2021-08-02 20:20:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46115) 2021-08-02 20:20:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46109) 2021-08-02 20:20:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46110) 2021-08-02 20:25:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46108) 2021-08-02 20:25:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46109) 2021-08-02 20:25:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46114) 2021-08-02 20:25:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 20:25:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46112) 2021-08-02 20:25:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46111) 2021-08-02 20:25:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46115) 2021-08-02 20:25:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46113) 2021-08-02 20:30:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46114) 2021-08-02 20:30:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 20:30:00+00:00 USD_CHF  NOTHING 

(pid=46110) 2021-08-02 21:35:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46109) 2021-08-02 21:35:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46111) 2021-08-02 21:35:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 21:35:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46108) 2021-08-02 21:35:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46111) 2021-08-02 21:40:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46113) 2021-08-02 21:40:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46110) 2021-08-02 21:40:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46109) 2021-08-02 21:40:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46113) 2021-08-02 21:45:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46111) 2021-08-02 21:45:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 21:45:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46110) 2021-08-02 21:45:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46108) 2021-08-02 21:45:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46109) 2021-08-02 21:45:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46114) 2021-08-02 21:45:00+00:00 GBP_USD  NOTHING 

(pid=46110) 2021-08-02 22:45:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 22:45:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46108) 2021-08-02 22:50:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46113) 2021-08-02 22:50:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46115) 2021-08-02 22:50:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46109) 2021-08-02 22:50:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46114) 2021-08-02 22:50:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46110) 2021-08-02 22:50:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 22:50:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46111) 2021-08-02 22:50:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46110) 2021-08-02 22:55:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 22:55:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46115) 2021-08-02 22:55:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46109) 2021-08-02 22:55:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46108) 2021-08-02 22:55:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46114) 2021-08-02 22:55:00+00:00 GBP_USD  NOTHING 

(pid=46110) 2021-08-02 23:55:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46112) 2021-08-02 23:55:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46115) 2021-08-02 23:55:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46108) 2021-08-02 23:55:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46113) 2021-08-02 23:55:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46111) 2021-08-02 23:55:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46108) 2021-08-03 00:00:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46110) 2021-08-03 00:00:00+00:00 AUD_USD  NOTHING HAPPENS
(pid=46112) 2021-08-03 00:00:00+00:00 USD_CHF  NOTHING HAPPENS
(pid=46115) 2021-08-03 00:00:00+00:00 USD_JPY  NOTHING HAPPENS
(pid=46109) 2021-08-03 00:00:00+00:00 GBP_JPY  NOTHING HAPPENS
(pid=46114) 2021-08-03 00:00:00+00:00 GBP_USD  NOTHING HAPPENS
(pid=46113) 2021-08-03 00:00:00+00:00 EUR_GBP  NOTHING HAPPENS
(pid=46111) 2021-08-03 00:00:00+00:00 EUR_USD  NOTHING HAPPENS
(pid=46108) 2021-08-03 00:05:00+00:00 GBP_CHF  NOTHING HAPPENS
(pid=46110) 2021-08-03 00:05:00+00:00 AUD_USD  NOTHING 

KeyboardInterrupt: 

In [5]:
ray.shutdown()